In [4]:
import gammapy 
import pyximport
import numpy as np
import astropy.units as u
import sys
from gammapy.modeling import Fit, Parameter, Parameters
from gammapy.modeling.models import Models
from gammapy.maps import MapAxis
from gammapy.modeling.models.spectral import scale_plot_flux
from gammapy.estimators import  FluxPointsEstimator
from gammapy.modeling.models import IRFModels, EffAreaIRFModel, ERecoIRFModel
    
sys.path.append('/home/katrin/Documents/nuisance_summary/')
sys.path.append('../../') 
from Dataset_Creation import sys_dataset   # noqa: E402
import Dataset_load  # noqa: E402


pyximport.install()

print(f'loaded gammapy version: {gammapy.__version__} ' )
print('Supposed to be 1.0 (21-12-2022)' )



scaled_amplitude = Parameter("amplitude", value=1e-12)
lambda_ = Parameter("lambda_", value=1/60)

dataset_asimov = Dataset_load.create_asimov(
    model="ecpl", source="PKSflare", parameters=Parameters([scaled_amplitude, lambda_])
)

mask = dataset_asimov.mask.data.sum(axis=2).sum(axis=1)>0

ebins = dataset_asimov.counts.geom.axes[0].center[mask]
print(len(ebins))


shift = 0.
tilt = 0.
resolution = 0.1
bias = 0.1

N = 500

save = True
save_flux = True
save_fluxpoints = False
save_fluxpoints_N = False
dataset_N = True


for n in range(N):
    print()
    print('====' * 30)
    print(n)
    print('====' * 30)
    
    res_rnd = np.random.normal(0, resolution, 1)
    bias_rnd = np.random.normal(0, bias, 1)
    
    print(f"res {res_rnd}, bias {bias_rnd}")
    sys_d_cor = sys_dataset(dataset_asimov= dataset_asimov,
                    shift = 0, 
                    tilt = 0,
                    resolution = res_rnd,
                    bias = bias_rnd,
                    rnd = True)
    dataset = sys_d_cor.create_dataset()
    fit_cor = Fit(store_trace=False)
    minuit_opts = {"tol": 0.001, "strategy": 2}
    fit_cor.optimize_opts = minuit_opts
    result_cor = fit_cor.run([dataset])


    if save:
        with open("data/7a_P_draw_info.txt", "a") as myfile:
            myfile.write(str(float(bias_rnd)) + '    '+ str(float(res_rnd)) + '    ' +  str(float(dataset.stat_sum())) + '\n')

    stri = ""
    for p in ['amplitude', 'index', 'lambda_', 'norm', 'tilt']:
        stri += str(dataset.models.parameters[p].value)  + '   ' +  str(dataset.models.parameters[p].error)  + '   '
    print(stri)
    if save:
        with open("data/7a_P_draw_par.txt", "a") as myfile:
            myfile.write(stri + '\n')



    fluxes = []
    for e in ebins:
        flux =  dataset.models[0].spectral_model(e)
        fluxes.append(flux.value)

    ff = str()
    for f in fluxes:
        ff += str(f) + "  "
    #print(ff)
    if save:
        with open("data/7a_P_draw_flux.txt", "a") as myfile:
            myfile.write( ff + '\n')


    energy_bounds = (ebins[0], ebins[-1] ) * u.TeV

    energy_min, energy_max = energy_bounds
    energy = MapAxis.from_energy_bounds(
        energy_min,
        energy_max,
        len(ebins),
    )

    fluxe2, _ = dataset.models[0].spectral_model._get_plot_flux(sed_type='dnde', energy=energy)
    fluxe2 = scale_plot_flux(fluxe2, energy_power=2)
    fluxe2 = fluxe2.quantity[:, 0, 0]
    fluxe2 = np.array(fluxe2)   
    ff = str()
    for f in fluxe2:
        ff += str(f) + "  "
    if save:
        with open("data/7a_P_draw_flux2e.txt", "a") as myfile:
            myfile.write( ff + '\n')

    energy_edges = dataset.geoms['geom'].axes[0].edges[::2]
    if save_fluxpoints:
        esti  = FluxPointsEstimator(energy_edges= energy_edges)
        fluxpoints = esti.run([dataset])
        fluxpoints.write(f'data/fluxpoints/6_fluxpoints_{bias_rnd[0]:.6}_{res_rnd[0]:.6}.fits')
        Models([dataset.models[0]]).write(f'data/fluxpoints/6_model_{bias_rnd[0]:.6}_{res_rnd[0]:.6}.fits')
        
    if dataset_N:
        dataset_N = sys_d_cor.create_dataset_N(e_reco_n = 10)
        zero = 1e-24
        dataset_N.models  = Models([dataset_N.models[0], dataset_N.background_model, IRFModels(eff_area_model = None,
                                                                                               e_reco_model = ERecoIRFModel(),
                                                                                              datasets_names=  dataset_N.name) ])
        # addional parameter bias and resolution (ereco) but are frozen
        penalising_invcovmatrix = np.zeros((2,2))
        # 'bias', 'resolution', 'norm_nuisance',  'tilt_nuisance',
        np.fill_diagonal(
            penalising_invcovmatrix,
            [ 1 / bias**2, 1 / resolution**2],
        )
        dataset_N.penalising_invcovmatrix = penalising_invcovmatrix
        dataset_N.irf_model.e_reco_model.parameters['bias'].frozen = False
        dataset_N.irf_model.e_reco_model.parameters['resolution'].frozen = False

        fit_cor = Fit(store_trace=False)
        result_cor = fit_cor.run([dataset_N])
        
        stri = ""
        for p in ['amplitude', 'index', 'lambda_', 'norm', 'tilt', 'bias', 'resolution']:
            stri += str(dataset_N.models.parameters[p].value)  + '   ' +  str(dataset_N.models.parameters[p].error)  + '   '
        print(stri)
        if save:
            with open("data/7a_N_P_draw_par.txt", "a") as myfile:
                myfile.write(stri + '\n')



        fluxes = []
        for e in ebins:
            flux =  dataset_N.models[0].spectral_model(e)
            fluxes.append(flux.value)

        ff = str()
        for f in fluxes:
            ff += str(f) + "  "
        #print(ff)
        if save:
            with open("data/7a_N_P_draw_flux.txt", "a") as myfile:
                myfile.write( ff + '\n')


        energy_bounds = (ebins[0], ebins[-1] ) * u.TeV

        energy_min, energy_max = energy_bounds
        energy = MapAxis.from_energy_bounds(
            energy_min,
            energy_max,
            len(ebins),
        )

        fluxe2, _ = dataset_N.models[0].spectral_model._get_plot_flux(sed_type='dnde', energy=energy)
        fluxe2 = scale_plot_flux(fluxe2, energy_power=2)
        fluxe2 = fluxe2.quantity[:, 0, 0]
        fluxe2 = np.array(fluxe2)   
        ff = str()
        for f in fluxe2:
            ff += str(f) + "  "
        if save:
            with open("data/7a_N_P_draw_flux2e.txt", "a") as myfile:
                myfile.write( ff + '\n')
        
        
        if save_fluxpoints_N:
           
            esti  = FluxPointsEstimator(energy_edges= energy_edges)
            fluxpoints_N = esti.run([dataset_N])
            fluxpoints_N.write(f'data/fluxpoints/6_fluxpoints_N_{bias_rnd[0]:.6}_{res_rnd[0]:.6}.fits')
            Models([dataset_N.models[0]]).write(f'data/fluxpoints/6_model_N_{bias_rnd[0]:.6}_{res_rnd[0]:.6}.yaml')


loaded gammapy version: 0.1.dev19174+g56b5bdf 
Supposed to be 1.0 (21-12-2022)
loaded dataset:
/home/wecapstor1/caph/mppi045h/nuisance_summary/PKS_flare/HESS_public/dataset-simulated-ecpl.fits.gz
20

0
res [0.1212879], bias [0.00780496]
9.87264513911512e-13   5.641387412101425e-14   2.2654649506829103   0.075882733092463   0.026201290379591555   0.015765831898350017   1.0000161156986629   0.0015603687467771303   0.0   0.0   
1.0044930091373699e-12   5.6668318276686525e-14   2.3236071641304994   0.07938410369078025   0.0217176988293191   0.016460011200461224   0.999597157430102   0.0015644969289498507   0.0031167719957357594   0.0016937151273459152   0.04445372619835813   0.02734893913524786   -0.00015602699625478128   0.08273275638096968   

1
res [-0.02106403], bias [0.02216896]
1.1050050040383803e-12   5.859502715575514e-14   2.3121359003221653   0.07049024043340522   0.02414815973764547   0.01593982714764791   0.9994717674278223   0.0015601572228877458   0.0   0.0   
8.7794983352093

ValueError: cannot convert float NaN to integer